В качестве финального проекта реализована система подсказок похожих вопросов на данных сайта Quora. Поиск производится исключительно по основному заголовку без уточняющих деталей.

Система представлена микросервисом на основе Flask. Верхнеуровнево пайплайн и критерии можно представить так:

Сначала происходит фильтрация запроса по языку (с помощью библиотеки LangDetect) — исключаются все запросы, для которых определённый язык не равняется "en". Затем происходит поиск вопросов-кандидатов с помощью FAISS (по схожести векторов) — в этой части предлагается ограничиться векторизацией только тех слов, эмбеддинги которых есть в исходных GLOVE-векторах. Эти кандидаты реранжируются KNRM-моделью, после чего до 10 кандидатов выдаются в качестве ответа.


На сервере реализовано две ручки: для запросов (для поиска похожих вопросов) и для создания FAISS-индекса.

/query — принимает POST-запрос. Должна вернуть json, где status='FAISS is not initialized!' в случае, если в решение не были загружены вопросы для поиска с помощью второго метода. 

Формат запроса для query:

json-запрос, с единственным ключом 'queries', значение которого — список строк с вопросами (Dict[str, List[str]]).

Формат ответа (в случае созданного индекса) — json с двумя полями. lang_check описывает, был ли распознан запрос как английский (List[bool], True/False-значения), suggestions — List[Optional[List[Tuple[str, str]]]].

В этом списке для каждого запроса из query необходимо указать список (до 10) найденных схожих вопросов, где каждый вопрос представлен в виде Tuple, в котором первое значение — id текста (см. ниже), второе — сам непредобработанный текст схожего вопроса. Если проверка на язык не пройдена (не английский), либо произошёл какой-то сбой в обработке — оставьте None в списке вместо ответа (например, [[(..., ...), (..., ...), ...], None, ... ]).

/update_index — принимает POST-запрос, в котором в json присутствует поле documents, Dict[str,str] — все документы, где ключ — id текста, значение — сам текст. На предобработку и создание индекса даётся 200 секунд. Подразумевается, что инициализация происходит единоразово, поэтому не нужно беспокоиться о повторном вызове этого метода. В возвращаемом json'е должно быть два ключа: status (ok, если всё прошло гладко) и index_size, значение которого — единственное целое число, хранящее количество документов в индексе.

В ноутбуке реализована демонстрация работы сервиса. 

In [1]:
from IPython.display import display, HTML
display(HTML("<style>:root { --jp-notebook-max-width: 95% !important; }</style>"))

In [2]:
user_query = input("Quora question (Is it possible to breathe through one's butt? as example):")

Quora question (Is it possible to breathe through one's butt? as example): Is it possible to breathe through one's butt?


In [5]:
import pandas as pd, numpy as np
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)
pd.set_option('display.max_colwidth', 255)

In [6]:
import requests, json
from typing import Dict, List, Tuple, Union, Callable, Optional
import builder 
# URL Flask app
base_url = 'http://127.0.0.1:5000'

class Solution:
  # import QUORA data + debug functions 
  # generate query, generate data for initial  FAISS indexing
  def __init__(
      self,
      glue_qqp_dir,
      ):
    self.glue_qqp_dir = glue_qqp_dir
    self.glue_df_train = self.get_glue_df('train')
    self.glue_df_test = self.get_glue_df('dev')
    self.json_query = self._generate_json_server_query()
    self.json_documents = self._generate_json_server_documents()

  def get_glue_df(self, partition_type: str) -> pd.DataFrame:
      assert partition_type in ['dev', 'train']
      if pd.__version__ > '1.3.0':
        glue_df = pd.read_csv(
            self.glue_qqp_dir + f'/{partition_type}.tsv', sep='\t', on_bad_lines='skip', dtype=object)
      else:
        glue_df = pd.read_csv(
            self.glue_qqp_dir + f'/{partition_type}.tsv', sep='\t', error_bad_lines=False, dtype=object)
      glue_df = glue_df.dropna(axis=0, how='any').reset_index(drop=True)
      glue_df_fin = pd.DataFrame({
          'id_left': glue_df['qid1'],
          'id_right': glue_df['qid2'],
          'text_left': glue_df['question1'],
          'text_right': glue_df['question2'],
          'label': glue_df['is_duplicate'].astype(int)
      })
      return glue_df_fin

  def _generate_json_server_query(self) -> Dict[str, List[str]]:
    # generating query for debug
    return {'query' : self.glue_df_test['text_left'].sample(10).tolist() + ['Этот запрос на другом языке', user_query]}

  def _generate_json_server_documents(self) -> Dict[str, Dict[str, List[str]]]:
    # generating documents for debug
    left_dict = self.glue_df_train[['id_left', 'text_left']].set_index('id_left').to_dict()['text_left']
    # right_dict = self.glue_df_train[['id_right', 'text_right']].set_index('id_right').to_dict()
    # documents = left_dict.update(right_dict)
    return {'documents' : left_dict}   
  

def query() -> Tuple:
    # return random queries from QUORA test.tsv dataset
    url = f'{base_url}/query'
    payload = sol._generate_json_server_query()

    response = requests.post(url, data=json.dumps(payload), headers=headers)
    return response.json(), payload

def update_index() -> Dict:
   # return QUORA train.tsv text documents to init FAISS index
   url = f'{base_url}/update_index'

   payload = sol._generate_json_server_documents()
   response = requests.post(url, data=json.dumps(payload), headers=headers)
   return response.json()

if __name__ == '__main__':
    # init QUORA dataset
    sol = Solution(builder.glue_qqp_dir)
    headers = {'Content-Type': 'application/json'}
    
    # POST query on server, check without FAISS index update
    query_response, _ = query()
    print('Query Response: ',  query_response)
    
    # POST documents on server to init FAISS indexes
    update_response = update_index()
    print('Update Response: ', update_response)
    
    # POST query on server
    query_response, query = query()
    print('Query Response: ',  query_response)

Query Response:  {'lang_check': [True, True, True, True, True, True, True, True, True, True, False, True], 'suggestions': [[['72228', 'Why surgical masks have a blue and white side?'], ['126045', 'Why are most of the flags red, blue and white?'], ['123180', 'Brown shoes with a navy suit, yes or no?'], ['118732', 'What might cause a blue screen on a Macbook?'], ['254119', 'Can the White House be vulnerable to a blackout?'], ['208910', 'Can a black or Asian person have blue eyes?'], ['174862', 'Why is white colour only showing once in the spectrum?'], ['419477', "What are the best men's white tube socks?"], ['70129', 'Is it possible to make a black colour LED?'], ['352933', 'What is the difference between black holes and black materials?']], [['145465', "What could be the possible reasons of Dhoni's decision of stepping down from captaincy?"], ['529204', 'What would be different in India today if we had not got independence?'], ['373156', 'What one past personal decision would HM The Que

In [7]:
# Display query and ranked by KNRM suggestion from server 
suggestion_arr = np.array(query_response['suggestions'])[:,:,1]
N = suggestion_arr.shape[1] # count ranked documents for each query
query_arr =  np.array(query['query']).reshape(-1,1)

query_df = pd.DataFrame(np.repeat(query_arr, N), columns = ['query'])
suggestion_df = pd.DataFrame(suggestion_arr.reshape(-1), columns = ['suggestions'])

pd.concat([query_df, suggestion_df], axis = 1).set_index(['query', 'suggestions'])

Empty DataFrame
Columns: []
Index: [(What category of porn do girls like the most?, What type of boys do most teen girls like?), (What category of porn do girls like the most?, What are some of the best porn movies?), (What category of porn do girls like the most?, What types of woman nightwears the best for girls?), (What category of porn do girls like the most?, What types of boys does every girl like?), (What category of porn do girls like the most?, What are some of the best horror movies?), (What category of porn do girls like the most?, What are the best movies of all time?), (What category of porn do girls like the most?, What are some of the best erotic movies?), (What category of porn do girls like the most?, What are some of the best zombie movies?), (What category of porn do girls like the most?, What are some of the best sex stories?), (What category of porn do girls like the most?, What are some of the best teen clothing websites?), (What if the iberian union never fell?, What if the Roman Empire Never Fell?), (What if the iberian union never fell?, What if the Soviet Union never collapsed?), (What if the iberian union never fell?, What is wrong with the Philippines?), (What if the iberian union never fell?, What word has changed the world?), (What if the iberian union never fell?, What does it mean in Korean?), (What if the iberian union never fell?, Why the rinnegan is stronger than the sharingan ?), (What if the iberian union never fell?, What does のだ and んだ mean in Japanese?), (What if the iberian union never fell?, Could the British Empire come back?), (What if the iberian union never fell?, Are we on the verge of WW3?), (What if the iberian union never fell?, What's it like to retire early?), (How did Hillary Clinton lose the 2016 US Presidential Election despite winning the popular vote?, How could Hillary Clinton win the popular vote, but still lose the election?), (How did Hillary Clinton lose the 2016 US Presidential Election despite winning the popular vote?, How did Hillary Clinton win the popular vote but Donald Trump win the election?), (How did Hillary Clinton lose the 2016 US Presidential Election despite winning the popular vote?, How did Hillary Clinton mess up her chances of winning the US presidential election?), (How did Hillary Clinton lose the 2016 US Presidential Election despite winning the popular vote?, What are Hillary Clinton's chances of winning in the 2016 US Presidential Election?), (How did Hillary Clinton lose the 2016 US Presidential Election despite winning the popular vote?, Why should one vote for Hillary Clinton in the 2016 presidential election?), (How did Hillary Clinton lose the 2016 US Presidential Election despite winning the popular vote?, What is the likelihood of Donald Trump defeating Hillary Clinton and winning the 2016 US Presidential election?), (How did Hillary Clinton lose the 2016 US Presidential Election despite winning the popular vote?, Could Hillary Clinton ask for a recount in the 2016 US Presidential Election?), (How did Hillary Clinton lose the 2016 US Presidential Election despite winning the popular vote?, If Hillary Clinton won the popular vote, why did Donald Trump win the Electoral College?), (How did Hillary Clinton lose the 2016 US Presidential Election despite winning the popular vote?, If Hillary Clinton wins the presidential election 2016, how would it turn out for India?), (How did Hillary Clinton lose the 2016 US Presidential Election despite winning the popular vote?, What's your feeling on the fact Hillary Clinton won popular vote while losing the election?), (If you could write a letter to any fictional character, what would you write?, If you write a biography of someone, do you have to pay them something?), (If you could write a letter to any fictional character, what would you write?, If you could get everybody to read one book, what would it be?), (If you could write a letter to any fictional character, what would you write?, What is l